# Building your first AzureML web service using Python 

This tutorial demonstrates how to deploy a machine learning model built using Python and scikit-learn. 

Note: If you are using a DSVM, it must be configured as specified in the README in the [Machine Learning Operationalization](https://aka.ms/o16ncli) GitHub repo.

The tutorial uses the digits dataset that is part of the [scikit-learn distribution](http://scikit-learn.org/stable/tutorial/basic/tutorial.html).

In the tutorial, you will load a dataset, train a model on the dataset, and then publish a
realtime scoring API for the model.

To complete this tutorial:

* Sign in to your DSVM.
* Copy this notebook to a folder on the DSVM.
* In a browser, open https://&lt;your dsvm ip address&gt;:&lt;port&gt; and sign into the Jupyter server.
    * On a Linux DSVM the port is typically 8000
    * On a Windows DSVM the port is typically 9999
* In Jupyter, navigate to the folder and open the notebook.


In [1]:
# Import Azure ML API SDK. The SDK is installed implicitly with the latest
# version of the CLI in your default python environment
from azure.ml.api.schema.dataTypes import DataTypes
from azure.ml.api.schema.sampleDefinition import SampleDefinition
from azure.ml.api.realtime.services import prepare

In [2]:
# Read in the digits dataset
from sklearn import datasets
digits = datasets.load_digits()
print(digits.data)

[[  0.   0.   5. ...,   0.   0.   0.]
 [  0.   0.   0. ...,  10.   0.   0.]
 [  0.   0.   0. ...,  16.   9.   0.]
 ..., 
 [  0.   0.   1. ...,   6.   0.   0.]
 [  0.   0.   2. ...,  12.   0.   0.]
 [  0.   0.  10. ...,  12.   1.   0.]]


## Train your model

The task is to predict which digit an image represents in the digits dataset. There are samples of each of the 10 possible classes (the digits zero through nine) on which you *fit* an estimator to predict the classes to which unseen samples belong.

In [3]:
# Train an SVM classifier
from sklearn import svm
clf = svm.SVC(gamma=0.001, C=100.)
clf.fit(digits.data[:-1], digits.target[:-1])

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [4]:
# Predict new inputs
clf.predict(digits.data[-1:])[0]

8

### Save your model

Once you have a model that performs well, you can package it into a scoring service. To prepare for this, save your model locally. You then use the Joblib library to pickle the model.

In [5]:
# Save model
from sklearn.externals import joblib
import os
if not os.path.exists('sklearn'):
    os.makedirs('sklearn')
joblib.dump(clf, 'sklearn/model.pkl')
print("Model saved")

Model saved


## Authoring a Realtime Web Service

In this section, you author a realtime web service that scores the model you saved above. 

### Define the ```init``` and ```run``` functions

Start by defining your ```init``` and ```run``` functions in the cell below. 

The ```init``` function initializes the web service, loading in any data or models that it needs to score your inputs. In the example below, it loads in the trained model and the schema of your dataset.

The ```run``` function defines what is executed on a scoring call. In this simple example, the service loads the json input as a data frame and runs the pipeline on the input.

In [6]:
#The init and run functions will load and score your input using the saved model.
#They will also be used to generate the main.py script which will be part of your create service call.
def init():   
    # read in the model file
    from sklearn.externals import joblib
    global model
    model = joblib.load('sklearn/model.pkl')
        
def run(input_array):
    import json        
    if (input_array.shape != (1, 64)):
        return 'Bad input: Expecting a json encoded list of lists of shape (1,64).'
    else:
        pred = model.predict(input_array)[0]
        return json.dumps(str(pred))

### Test the ```init``` and ```run``` functions

Before publishing the web service, you can test the init and run functions in the notebook by running the the following cell.

In [12]:
init()
run(digits.data[-1:])

'"8"'

### Create Web Service Schema

1) Using the "prepare" function below automatically creates a schema JSON file for the web service.
2) It also creates a main.py file. 
Both of these files will be written the output folder (e.g. output_timestamp) which will be in the response of the prepare method below. 
You will need to provide the path to the JSON schema file and main.py when calling the create service command.

In [13]:
dataarray = digits.data[-1:]
inputs = {"input_array": SampleDefinition(DataTypes.NUMPY, dataarray)}
#Note the output location which is in the response of this call
#The output location will include the
prepare(run_func=run, init_func=init, input_types=inputs)

processing inputs
processing init function
processing run function
setting up output directory
Done setting up output directory, available here output_20170711205031


'output_20170711205031'

### Use the Azure Machine Learning CLI to deploy and manage your web services

From the commandline, run the following commands to deploy your service locally.

NOTE: Before you can run the Azure Machine Learning CLI command to operationalize the model, you must have set the environment variables that you generated when you setup your DSVM. If you did not do so when you configured the DSVM, see the README in the [Machine Learning Operationalization](https://aka.ms/o16ncli) GitHub repo for more information.

Open the folder containing the notebook, and navigate to output folder created by the running "prepare" in the previous cell.

**Local deployment to your machine**

Next, run the following commands to create the web service (Sklearn folder contains the model):

```
az ml env local
az ml service create realtime -f main.py -d ../sklearn -s service_schema.json -n mydigitservice -r scikit-py
```

**Deployment to a cluster (for production)**

To create and run the web service on the ACS cluster, change to the cluster mode and rerun the service creation:

```
az ml env cluster
az ml service create realtime -f main.py -d ../sklearn -s service_schema.json -n myremoteskdigitsservice -r scikit-py
```

To test the web service, run the following command with a sample data input:

```
az ml service run realtime -n mydigitservice -d "{\"input_array\": [[0.0, 0.0, 10.0, 14.0, 8.0, 1.0, 0.0, 0.0, 0.0, 2.0, 16.0, 14.0, 6.0, 1.0, 0.0, 0.0, 0.0, 0.0, 15.0, 15.0, 8.0, 15.0, 0.0, 0.0, 0.0, 0.0, 5.0, 16.0, 16.0, 10.0, 0.0, 0.0, 0.0, 0.0, 12.0, 15.0, 15.0, 12.0, 0.0, 0.0, 0.0, 4.0, 16.0, 6.0, 4.0, 16.0, 6.0, 0.0, 0.0, 8.0, 16.0, 10.0, 8.0, 16.0, 8.0, 0.0, 0.0, 1.0, 8.0, 12.0, 14.0, 12.0, 1.0, 0.0]]}"
```

To get the sample input data for the test call, use the following command while in local mode (requires the web service to have been deployed locally):

```
az ml service view realtime -n mydigitservice
```